In [1]:
!wget https://s202.q4cdn.com/436759741/files/doc_financials/2023/q4/Exhibit99_Q4_2023-c.pdf

--2024-10-23 11:42:46--  https://s202.q4cdn.com/436759741/files/doc_financials/2023/q4/Exhibit99_Q4_2023-c.pdf
Resolving s202.q4cdn.com (s202.q4cdn.com)... 181.41.142.154
Connecting to s202.q4cdn.com (s202.q4cdn.com)|181.41.142.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 823542 (804K) [application/pdf]
Saving to: ‘Exhibit99_Q4_2023-c.pdf’

Exhibit99_Q4_2023-c 100%[===================>] 804.24K  2.85MB/s    in 0.3s    

2024-10-23 11:42:46 (2.85 MB/s) - ‘Exhibit99_Q4_2023-c.pdf’ saved [823542/823542]



In [2]:
###################################
########### INGESTION #############
###################################

In [34]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from unstructured.partition.utils.constants import PartitionStrategy


loader = UnstructuredPDFLoader(
    file_path="./Exhibit99_Q4_2023-c.pdf",
    strategy=PartitionStrategy.HI_RES,
    chunking_strategy="by_title",
    new_after_n_chars=4000,  # Soft-max
    max_characters=4000,  # Hard-max
    combine_text_under_n_chars=2000,  # Combine chunks of < 200 chars
    mode='elements',  # Split the documents into elements such as Title and NarrativeText.
)
data = loader.load()

In [35]:
len(data)

32

In [49]:
data[0].__dict__

{'page_content': '<epam>\n\nEPAM Reports Results for Fourth Quarter and Full Year 2023\n\nFourth Quarter 2023\n\n• Revenues of $1.157 billion, down 6.0% year-over-year\n\n• GAAP Income from Operations was 10.6% of revenues and Non-GAAP Income from Operations was 17.3% of revenues\n\n• GAAP Diluted EPS of $1.66, a decrease of 36.4%, and Non-GAAP Diluted EPS of $2.75, a decrease of 6.1% on a year-over-year basis\n\nFull Year 2023\n\n• Revenues of $4.691 billion, down 2.8% year-over-year\n\n• GAAP Income from Operations was 10.7% of revenues and Non-GAAP Income from Operations was 16.3% of revenues\n\n• GAAP Diluted EPS of $7.06, a decrease of 0.4%, and Non-GAAP Diluted EPS of $10.59, a decrease of 2.8% on a year-over-year basis\n\nNewtown, PA, USA, February 15, 2024 — EPAM Systems, Inc. (NYSE: EPAM), a leading digital transformation services and product engineering company, today announced results for its fourth quarter and full year ended December 31, 2023.\n\n“EPAM\'s performance in 20

In [50]:
[doc.metadata['category'] for doc in data]

['CompositeElement',
 'Table',
 'Table',
 'CompositeElement',
 'Table',
 'CompositeElement',
 'CompositeElement',
 'CompositeElement',
 'CompositeElement',
 'CompositeElement',
 'Table',
 'CompositeElement',
 'Table',
 'CompositeElement',
 'Table',
 'CompositeElement',
 'Table',
 'CompositeElement',
 'Table',
 'CompositeElement',
 'Table',
 'CompositeElement',
 'Table',
 'CompositeElement',
 'CompositeElement',
 'CompositeElement',
 'Table',
 'CompositeElement',
 'Table',
 'CompositeElement',
 'Table',
 'CompositeElement']

In [38]:
data[2].page_content  # Table

'Americas EMEA APAC GEE* S676M $454M S26M S1M -7.6% | -0.3%1 -10.9% 1 -91.6% J YoY YoY YoY YoY'

In [39]:
###################################
########### RETRIEVER #############
###################################

In [40]:
from dotenv import load_dotenv, find_dotenv
import os

env_path = find_dotenv()

if env_path:
    print(f".env file found at: {env_path}")
else:
    print("No .env file found.")

load_dotenv(dotenv_path=env_path)

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


.env file found at: /Users/Sanchit_Balchandani/workspace/python-ws/multimodal-rag/.env


In [41]:
# from getpass import getpass

# OPENAI_API_KEY = getpass('Enter OpenAI Key: ')

In [42]:
from langchain_openai import OpenAIEmbeddings


OPENAI_EMBEDDING_MODEL = OpenAIEmbeddings(model='text-embedding-ada-002', api_key=OPENAI_API_KEY)

In [43]:
from langchain_chroma import Chroma

vectorstore = Chroma(
    collection_name='OSM-21-Oct-2024-tradition-v2',
    embedding_function=OPENAI_EMBEDDING_MODEL,
    collection_metadata={"hnsw:space": "cosine"},
)

In [44]:
retriever = vectorstore.as_retriever()

In [52]:
from langchain.vectorstores.utils import filter_complex_metadata

# Filter complex metadata
data = filter_complex_metadata(data)
retriever.vectorstore.add_documents(data)

['4edb36da-12d4-444c-952c-3054d1988ebf',
 'b2e48879-086e-4c93-a8b5-2c4099367963',
 'e944eb7d-5692-4f9f-8ca5-2bbc3baaf67f',
 'bb25fbae-b0a5-4dc7-8480-51231b20864f',
 '6c130ca1-88f2-4279-9d90-f0d55f983847',
 'c64eaa46-d184-47c9-94d1-9772ba10ec1d',
 '0f0dade5-a83e-4828-b526-e2fe2ec0bbbb',
 '6b5ee132-a11d-413c-a1dd-ce1bcee28e22',
 '6b000e25-03db-4a9e-bb69-04afb803ff80',
 'e2b14184-e35b-46a5-80dc-77fb72750953',
 'df411f61-0361-4c12-a55f-3ade5b857d7f',
 'e40d3515-5c0f-42df-8425-01e6cc1c7c8a',
 'e68a4615-f076-4161-be9d-18a637007d0d',
 'ba192544-5e8a-4a5e-b9eb-baf55266a5df',
 '13458341-581c-414c-8290-47e0988ce664',
 'd94b8323-0e74-4131-b221-a248f8c0e210',
 '4822eb03-837c-4592-8bbe-659b2bad13b0',
 '98855a08-dbb2-4f4a-813b-54986bb9344f',
 '561da06a-407c-44ab-97f4-15116f74bf82',
 'af0b9e1d-3fdf-4500-bae0-c414918084c5',
 '3a5ad9ab-d0c0-40b5-ada7-4cd83d1c1332',
 '7dc12a51-38b8-4296-a5ee-a3c85d379147',
 '3bc7f44c-3ad7-45ea-9c4b-1f7711233ff1',
 '356496ae-8658-4a44-840f-fe4d0ed7828e',
 '7c80d7c8-3ee2-

In [53]:
vectorstore.get(include=["metadatas", "documents", "embeddings"])

{'ids': ['bb90e0e9-57e9-4224-bac3-d79d331de469',
  '3a7128ad-80e9-4879-8f3b-272c7b041641',
  'deb7bea0-3f8a-4437-81aa-0de7b3a4d1ed',
  '954ffb45-88b5-45a4-8c6a-199d0f0616bf',
  'f9b9b020-3404-4ea9-9d51-e5889d49785f',
  '62fdc31c-32dc-4ff6-a47e-eef17c6f2177',
  '0bd6f7a4-3c6c-4f88-8586-5a0a5e88cf3b',
  '6ac0d00f-83e5-48b2-b043-32476b726147',
  'a6be0c84-e50f-49c7-8981-cc6cf56e4bbb',
  '9905520b-ffab-4530-8eb5-d72be754b0aa',
  '648bac7e-eb64-4a1c-b299-3cce3fabe2e3',
  '9364b399-08ad-43ee-9bfd-273654ce1c62',
  'f9237033-c331-48ea-a3f7-25e1c3a4364b',
  'eacaa7dc-029a-478c-9ecc-2b2ec8fa75c0',
  '43579103-5aba-4a38-92ad-76bfc3dd38bc',
  '9f94e0d0-caaf-409f-b6e6-721420d50a47',
  'ba1978a5-4450-4e56-aa06-c3433cd2f529',
  '376a6f13-c4d3-4dc2-8d0a-872099629768',
  '4ca074c9-27ba-42c6-84af-fd3c45bb1fd9',
  '309e5438-24d1-4c07-ad7b-42cd80749bd0',
  '4ec36a76-f2b3-48c2-8c18-a231366bac97',
  '0a9d7edc-fa18-47ab-8c08-dc122cae8543',
  'd3097fb8-98f4-4083-9924-14b9342e1cd2',
  'a1d312f3-971f-4f6c-815d-

In [54]:
vectorstore.get()

{'ids': ['bb90e0e9-57e9-4224-bac3-d79d331de469',
  '3a7128ad-80e9-4879-8f3b-272c7b041641',
  'deb7bea0-3f8a-4437-81aa-0de7b3a4d1ed',
  '954ffb45-88b5-45a4-8c6a-199d0f0616bf',
  'f9b9b020-3404-4ea9-9d51-e5889d49785f',
  '62fdc31c-32dc-4ff6-a47e-eef17c6f2177',
  '0bd6f7a4-3c6c-4f88-8586-5a0a5e88cf3b',
  '6ac0d00f-83e5-48b2-b043-32476b726147',
  'a6be0c84-e50f-49c7-8981-cc6cf56e4bbb',
  '9905520b-ffab-4530-8eb5-d72be754b0aa',
  '648bac7e-eb64-4a1c-b299-3cce3fabe2e3',
  '9364b399-08ad-43ee-9bfd-273654ce1c62',
  'f9237033-c331-48ea-a3f7-25e1c3a4364b',
  'eacaa7dc-029a-478c-9ecc-2b2ec8fa75c0',
  '43579103-5aba-4a38-92ad-76bfc3dd38bc',
  '9f94e0d0-caaf-409f-b6e6-721420d50a47',
  'ba1978a5-4450-4e56-aa06-c3433cd2f529',
  '376a6f13-c4d3-4dc2-8d0a-872099629768',
  '4ca074c9-27ba-42c6-84af-fd3c45bb1fd9',
  '309e5438-24d1-4c07-ad7b-42cd80749bd0',
  '4ec36a76-f2b3-48c2-8c18-a231366bac97',
  '0a9d7edc-fa18-47ab-8c08-dc122cae8543',
  'd3097fb8-98f4-4083-9924-14b9342e1cd2',
  'a1d312f3-971f-4f6c-815d-

In [55]:
vectorstore._collection.count()

64

In [56]:
query = "Give me the revenue for financial services for year 2024?"
docs = retriever.invoke(query, limit=5)

In [57]:
[doc.page_content for doc in data]

['<epam>\n\nEPAM Reports Results for Fourth Quarter and Full Year 2023\n\nFourth Quarter 2023\n\n• Revenues of $1.157 billion, down 6.0% year-over-year\n\n• GAAP Income from Operations was 10.6% of revenues and Non-GAAP Income from Operations was 17.3% of revenues\n\n• GAAP Diluted EPS of $1.66, a decrease of 36.4%, and Non-GAAP Diluted EPS of $2.75, a decrease of 6.1% on a year-over-year basis\n\nFull Year 2023\n\n• Revenues of $4.691 billion, down 2.8% year-over-year\n\n• GAAP Income from Operations was 10.7% of revenues and Non-GAAP Income from Operations was 16.3% of revenues\n\n• GAAP Diluted EPS of $7.06, a decrease of 0.4%, and Non-GAAP Diluted EPS of $10.59, a decrease of 2.8% on a year-over-year basis\n\nNewtown, PA, USA, February 15, 2024 — EPAM Systems, Inc. (NYSE: EPAM), a leading digital transformation services and product engineering company, today announced results for its fourth quarter and full year ended December 31, 2023.\n\n“EPAM\'s performance in 2023 reflects our 

In [21]:
###################################
########### Synthesis #############
###################################

In [22]:
from langchain_openai import ChatOpenAI

CHAT_MODEL = ChatOpenAI(model_name='gpt-4o-mini', api_key=OPENAI_API_KEY, temperature=0)

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context 
to answer the question. If you don't know the answer, just say that you don't know.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | CHAT_MODEL
    | StrOutputParser()
)

In [25]:
query = "Give me the revenue for financial services for year 2024?"
response = rag_chain.invoke(query)

In [26]:
response

"The retrieved context does not provide specific revenue figures for financial services for the year 2024. Therefore, I don't know the answer."

In [27]:

query = query = "What are the assets EPAM hold for 2022, 2023"
response = rag_chain.invoke(query)

In [28]:
response

"The retrieved context does not provide specific information about the total assets held by EPAM for the years 2022 and 2023. Therefore, I don't know the answer."